In [16]:
# read in csv with dca mappings
# read in MSA
# for each seq in the MSA, 
# index, seqid, MSA_i, MSA_j, seq_i, seq_j, score

#a--b-c--de
#abcde
# seq_i = MSA_i - num_gaps_before
# MSA_i = seq_i + num_gaps_before

In [1]:
from Bio import AlignIO

In [2]:
def dca_to_msa(msa, dca_output_file, mapped_output_file, type = 'PLM'):

    # read coupling data from csv
    coupling_data = []

    for line in dca_output_file:
        datapt = line.split(',')
        coupling_data.append(datapt)

    # coupling_data
    # seq_mapped_data = []

    for (i, record) in enumerate(msa):
        # seq_i = MSA_i - num_gaps[MSA_i]

        # computes number of gaps occurring before and including index i
        n = 0
        num_gaps = []
        record_str = str(record.seq)

        for c in record_str:
            if c == '-':
                n += 1
                num_gaps.append(-1)
            else:
                num_gaps.append(n)
        for coupling in coupling_data:
            # coupling indices are 1-indexed

            if type == 'PLM':
                MSA_i = int(coupling[0]) - 1
                MSA_j = int(coupling[1]) - 1
            elif type == 'MF':
                MSA_i = int(coupling[0])
                MSA_j = int(coupling[1])

            DCA_score = coupling[2]

            if num_gaps[MSA_i] == -1 or num_gaps[MSA_j] == -1:
                continue

            seq_i = MSA_i - num_gaps[MSA_i]
            seq_j = MSA_j - num_gaps[MSA_j]

            if type == 'PLM':
                csv_datapt_str = "{},{},{},{},{},{},{}".format(i, record.id, MSA_i + 1, MSA_j + 1, seq_i, seq_j, DCA_score)
            elif type == 'MF':
                csv_datapt_str = "{},{},{},{},{},{},{}".format(i, record.id, MSA_i, MSA_j, seq_i, seq_j, DCA_score)
                
            # elif type == 'MF':
            #     MSA_i = int(coupling[0])
            #     MSA_j = int(coupling[1])

            # MSA indices are 1-indexed, but seq indices are 0-indexed!!!
            # index, seqid, MSA_i, MSA_j, seq_i, seq_j, score
            
            mapped_output_file.write(csv_datapt_str)

        

## PLM-DCA

In [ ]:
cadherin_msa_path = "../data/cadherin/Trimmed_PF00028_plmdca.fa"
cadherin_csv_path = "../data/cadherin/PF00028_plmdca.csv"
cadherin_mapped_csv_path = "../data/cadherin/PF00028_plmdca_mapped.csv"

cadherin_msa = AlignIO.read(cadherin_msa_path, 'fasta')
cadherin_csv_file = open(cadherin_csv_path, 'r')
cadherin_mapped_csv_file = open(cadherin_mapped_csv_path, 'w')

dca_to_msa(cadherin_msa, cadherin_csv_file, cadherin_mapped_csv_file)

## MF-DCA

In [6]:
cadherin_msa_path = "../data/cadherin/PF00028.alignment.seed"
cadherin_csv_path = '../data/cadherin/PF00028_mfdca.csv'
cadherin_mapped_csv_path = '../data/cadherin/PF0028_mfdca_mapped.csv'

cadherin_msa = AlignIO.read(cadherin_msa_path, 'stockholm')
cadherin_csv_file = open(cadherin_csv_path, 'r')
cadherin_mapped_csv_file = open(cadherin_mapped_csv_path, 'w')

dca_to_msa(cadherin_msa, cadherin_csv_file, cadherin_mapped_csv_file, type = 'MF')

In [23]:
import pandas as pd

df = pd.read_csv(cadherin_csv_path)
df['first_site'] = df['first_site'].astype(int)
df['second_site'] = df['second_site'].astype(int)

In [24]:
df

,first_site,second_site,mf_dca
0,0,1,0.028504
1,0,2,0.059369
2,0,3,0.034621
3,0,4,0.037939
4,0,5,0.032860
...,...,...,...
8251,125,127,0.047774
8252,125,128,0.027052
8253,126,127,0.013962
8254,126,128,0.027891


In [25]:
df.to_csv(cadherin_csv_path, index=False)